In [3]:
# import libraries
import requests
import pandas as pd
import time
import json
import pprint
from config import host_name, dbname, port, username, password, API_KEY
pp = pprint.PrettyPrinter(indent=2)

In [4]:
# youtube channel ID
CHANNEL_ID = "UCB2wtYpfbCpYDc5TeTwuqFA"

In [5]:
# API request 
pageToken = ""
url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000"+pageToken
response = requests.get(url).json()

In [5]:
pp.pprint(response)

{ 'etag': 's8TsWLTohQOMwbmZkgeyuTnDKyM',
  'items': [ { 'etag': '3notO30lESngMLKk1ugPfW29di4',
               'id': {'kind': 'youtube#video', 'videoId': 'hEqkeA6zddI'},
               'kind': 'youtube#searchResult',
               'snippet': { 'channelId': 'UCB2wtYpfbCpYDc5TeTwuqFA',
                            'channelTitle': 'Will Tennyson',
                            'description': 'Today I see if I have what it '
                                           'takes to survive an IFBB PRO '
                                           "female bodybuilder's brutal leg "
                                           'workout! Video includes lots and '
                                           'lots of ...',
                            'liveBroadcastContent': 'none',
                            'publishTime': '2022-01-27T15:00:27Z',
                            'publishedAt': '2022-01-27T15:00:27Z',
                            'thumbnails': { 'default': { 'height': 90,
                       

In [6]:
video_id = response['items'][0]['id']['videoId']
video_id

'hEqkeA6zddI'

In [7]:
video_title = response['items'][0]['snippet']['title']
video_title = str(video_title).replace('&amp;', '')
video_title = str(video_title).replace('&quot;', '')
video_title = str(video_title).replace('&#39;', "'")
video_title

'WHO’S STRONGER? | Male vs Female Bodybuilder'

In [8]:
# Grab the upload date
upload_date = response['items'][0]['snippet']['publishedAt']
upload_date = str(upload_date).split('T')[0] # remove the timestamp and timezone
upload_date

'2022-01-27'

In [9]:
def get_video_details(video_id):
       # collecting view, like and comment counts with a second API call
        url_video_stats = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&part=statistics&key='+API_KEY
        response_video_stats = requests.get(url_video_stats).json()
        view_count = response_video_stats['items'][0]['statistics']['viewCount']
        like_count = response_video_stats['items'][0]['statistics']['likeCount']
        comment_count = response_video_stats['items'][0]['statistics']['commentCount']
        
        return view_count, like_count, comment_count

In [10]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                video_title = str(video_title).replace("&#39;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [11]:
# create a dataframe
df2 = pd.DataFrame(columns= ['video_id', 'video_title', 'upload_date', 'view_count', 'like_count', 'comment_count'])
df2 = get_videos(df2)
df2.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,hEqkeA6zddI,WHO’S STRONGER? | Male vs Female Bodybuilder,2022-01-27,143494,8364,702
1,gSTHPqdBgUs,Should You &quot;EAT BIG TO GET BIG&quot;? (Cl...,2022-01-23,251544,13704,1931
2,pIkbRaz_pSU,I Tried The Worlds Fittest Kids Diet Workout,2022-01-20,521733,20550,1848
3,NCPPSAiTqgs,ZYZZ Diet Workout To Become The Aesthetic God,2022-01-16,405857,23804,2751
4,vbTcdBL9lGY,World’s Most Productive Day To Build Muscle (S...,2022-01-13,267220,12380,545


In [12]:
df2

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,hEqkeA6zddI,WHO’S STRONGER? | Male vs Female Bodybuilder,2022-01-27,143494,8364,702
1,gSTHPqdBgUs,Should You &quot;EAT BIG TO GET BIG&quot;? (Cl...,2022-01-23,251544,13704,1931
2,pIkbRaz_pSU,I Tried The Worlds Fittest Kids Diet Workout,2022-01-20,521733,20550,1848
3,NCPPSAiTqgs,ZYZZ Diet Workout To Become The Aesthetic God,2022-01-16,405857,23804,2751
4,vbTcdBL9lGY,World’s Most Productive Day To Build Muscle (S...,2022-01-13,267220,12380,545
...,...,...,...,...,...,...
242,8XLIYeWIGRs,I Tried Jason Momoas Workout Diet For AQUAMAN...,2019-07-18,1573140,27961,1975
243,6H-h5nzT4bI,I Lived Like Mark Wahlberg For A Whole Day | 4...,2019-07-13,1975574,59469,3286
244,2_CRP6CnnIg,Letting Celebrities DECIDE What i Eat for 24 H...,2019-07-09,242028,9048,345
245,k2Vj40tJ4kE,Letting Homeless People DECIDE What i Eat For ...,2019-07-01,119572,9225,384


In [13]:
# import the dataframe to an AWS database
!pip install psycopg2
import psycopg2 as ps


In [14]:
# Check for errors in connection to the database
def connect_to_db(host_name, dbname, port, username, password): 
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Beam me up, Scotty!')
    return conn

In [16]:
# conn to AWS db using imported credentials from config.py
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

Beam me up, Scotty!


In [ ]:
# create database table that looks like dataframe df2
df2

In [17]:
 # create database table, push command from notebook to my postgres DB on AWS
def create_table(curr):
    create_table_command = ('''CREATE TABLE IF NOT EXISTS videos (
                            video_id VARCHAR(255) PRIMARY KEY,
                            video_title TEXT NOT NULL,
                            upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                            view_count INTEGER NOT NULL,
                            like_count INTEGER NOT NULL,
                            comment_count INTEGER NOT NULL
                    )''')
    curr.execute(create_table_command) #executes SQL command on python notebook

In [18]:
curr = conn.cursor() # allows python to execute SQL commands and fetch the results from the db

In [19]:
create_table(curr)

In [20]:
# executes SQL command onto the database
conn.commit()

In [21]:
# building the function for updating the database

# handling data for scalability

# check to see if video exists
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update
            update_row(curr,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

# perform update on existing videos
# perform an insert on new videos
 

In [22]:
def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")

    curr.execute(query, (video_id,)) # python needs video_id to be expressed as a tuple
    return curr.fetchone() is not None # fetchone() returns a single row from a table

In [23]:
def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

In [24]:
new_vid_df = update_db(curr,df2)

In [26]:
# write SQL INSERT INTO command (just like the UPDATE command) to insert data into videos table
# wrap it into a function
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)

In [27]:
# append data from df to SQL db

def append_from_df_to_db(curr, df2):
    for i, row in df2.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])

In [28]:
new_vid_df = update_db(curr,df2)


In [ ]:
tennydata = df2.to_csv('C:/Users/davez/Desktop/Youtube_Project/Youtube_Project/tenndydata.csv', index=False)

In [30]:
#insert new videos into db table

append_from_df_to_db(curr, new_vid_df)
conn.commit()